**라이브러리 import**

### 라이브러리 설명

1. folium library  : 지도 위에 시각화를 할 수 있는 라이브러리
* Marker    
마커를 설정해줍니다  
 -color: 마커의 색깔  
 -icon: icon종류  
 -tooltip: 마우스 커서를 마커에 올렸을 때 뜨는 문구  
 -popup: 마우스로 마커를 클릭했을때 뜨는 문구 
  
* MarkerCluster
마커들을 묶음으로 보여줍니다.


2. geopy library: 장소의 주소, 위도, 경도, 고도, 거리 계산 등의 기능을 쉽게 이용할 수 있게 한 라이브러리
* geodesic

 특정 표면에서의 두 지점 사이 가장 짧은 거리를 계산해줍니다.

 즉, 지구에서 두 장소 사이의 거리를 계산해줍니다

In [ ]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import folium
from folium.plugins import MarkerCluster

**데이터 불러오기**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/파데분/파데분플젝/최종/데이터/shops.csv', sep=',')  #소상공인시장진흥공단 상가(상권)정보.csv를 shops.csv로 불러옵니다
columns = ['상호명', '상권업종대분류명','상권업종소분류명', 
           '시도명', '시군구명', '행정동명', '도로명주소', 
           '경도', '위도']
cond = df['시군구명'] == '서대문구' #본 프로그램은 서대문구의 경우만 이용합니다
df = df[cond]
dong = sorted(set(list((df['행정동명'])))) #서대문구의 동 목록을 정렬하여 가져옵니다

### **메인 프로그램**

folium 라이브러리를 활용해 음식점 위치를 아이콘으로 시각화하여 사용자에게 보여줍니다.  
사용자가 행정동명을 검색하면 해당 행정동명에 위치한 음식점을 사용자에 선택에 따라 2가지 모드(전체 뷰 모드, 검색 모드)로 보여줍니다.  

In [ ]:
print('서대문구의 행정동명은 다음과 같습니다 : \n', dong)

print()
region = input('행정동명을 입력해주세요: ')
while True:
    dfm= pd.read_csv('/content/gdrive/MyDrive/파데분/파데분플젝/최종/데이터/final_' + region + '.csv')
    df=pd.DataFrame(dfm.loc[:,'경도':'위도'],columns=['경도','위도'])
    print()
    mode = int(input('모드를 골라주세요: 1 - 해당 동 전체 뷰 모드, 2 - 검색 모드: , 3 - 프로그램 종료: '))


    if mode == 1:

        lat = df['위도'].mean()
        long = df['경도'].mean()
        m1 = folium.Map(location = [lat, long], zoom_start = 13)
        marker_cluster=MarkerCluster().add_to(m1)
        for i in dfm.index:
            sub_lat = df.loc[i,'위도']
            sub_long = df.loc[i,'경도']
        #Marker기능으로 tooltip, icon, popup을 지정하여 시각화 효과를 높이고 주소, 나와의 거리를 제공합니다
            folium.Marker([sub_lat, sub_long],
                          icon = folium.Icon(color='orange', icon = 'glyphicon-cutlery'),
                          tooltip = '<i><b>\'' + str(dfm.loc[i,'상호명']) + '\'</b></i>').add_to(marker_cluster)
        print('____________________________________________________')
        print('1번: 해당 동 전체 뷰 모드로 설정되었습니다.')
        print()
        print(region + '의 음식점 분포입니다')
        display(m1)

        
    elif mode == 2: #검색 모드의 경우, 나의 위치와 음식점을 입력해 원하는 음식점과의 거리를 알 수 있습니다
        print('____________________________________________________')
        print('2번: 검색 모드로 설정되었습니다')
        print()
        print('나의 위치를 입력해주세요: ')

        #나의 현재 위치를 위도와 경도 좌표로 입력합니다
        my_lat = float(input('위도를 입력해주세요(-90~90): ')) 
        my_long = float(input('경도를 입력해주세요(-180~180): ')) 
        print()
        my_location = (my_lat, my_long)

        #geodesic함수를 사용해 내 위치와, 입력한 동에 있는 모든 음식점 사이의 거리를 계산합니다
        df['distance']=df.apply(lambda x:geodesic((x['위도'],x['경도']),my_location),axis=1) #axis=1을 통해 열 방향으로 거리 계산을 수행하고, 그 결과를 df에 'distance' column으로 저장합니다

        #원하는 음식점 이름을 입력합니다
        store = input("주소와 나와의 거리를 알고 싶은 가게 이름을 입력하세요: ")
        print()
        
        #입력한 음식점의 정보를 뽑아냅니다
        ii=dfm.index[dfm['상호명'].str.contains(store)].tolist() #가게의 이름을 다 기억하지 못해 일부만 입력하더라도 작동하도록 합니다
        i=ii[0] #index로 사용할 수 있도록  i를 list의 요소로 가져옵니다
        store_location = (df['위도'][i],df['경도'][1]) #입력한 음식점의 위도, 경도 좌표를 저장합니다
        
        #folium 라이브러리의 Map기능으로 지도에 내 위치와 음식점의 위치를 표시합니다
        m2=folium.Map(location = [(store_location[0] + my_location[0]) / 2, (store_location[1] + my_location[1]) / 2], zoom_start=12)

        #Marker기능으로 tooltip, icon, popup을 지정하여 시각화 효과를 높이고 주소, 나와의 거리를 제공합니다
        folium.Marker(store_location, tooltip= '<b><i>'+str(dfm['상호명'][i])+'</b></i>',
                      icon=folium.Icon(color='blue', icon='search'),
                      popup='<i>\'도로명주소\'</i>' + '<br/>' + dfm['도로명주소'][i] +  '<br/>'*2 + 
                      '<i>\'별점\'</i>' + '<br/>' + str(dfm['naver_visitor_review_rate'][i])
                      + '<br/>'* 2  + '<i>\'나와의 거리\'</i>' + '<br/>' + str(df['distance'][i])).add_to(m2)
        folium.Marker(my_location, tooltip = '<i><b>\'나의 위치\'</b></i>', icon = folium.Icon(color = 'red', icon = 'user')).add_to(m2)
        print('<',dfm['상호명'][i],'>의 주소, 거리에 대한 정보입니다.')
        #시각화가 완성된 지도를 보여줍니다
        display(m2)
        folium.Map().get_root().render()
        
        
    elif mode == 3: #프로그램 종료를 선택할 경우 break를 통해 반복문을 종료합니다
        print()
        print('이용해 주셔서 감사합니다')
        break
      

서대문구의 행정동명은 다음과 같습니다 : 
 ['남가좌1동', '남가좌2동', '북가좌1동', '북가좌2동', '북아현동', '신촌동', '연희동', '천연동', '충현동', '홍은1동', '홍은2동', '홍제1동', '홍제2동', '홍제3동']

행정동명을 입력해주세요: 신촌동

모드를 골라주세요: 1 - 해당 동 전체 뷰 모드, 2 - 검색 모드: , 3 - 프로그램 종료: 2
____________________________________________________
2번: 검색 모드로 설정되었습니다

나의 위치를 입력해주세요: 
위도를 입력해주세요(-90~90): 36
경도를 입력해주세요(-180~180): 127

주소와 나와의 거리를 알고 싶은 가게 이름을 입력하세요: 돈불

< 돈불1971 >의 주소, 거리에 대한 정보입니다.



모드를 골라주세요: 1 - 해당 동 전체 뷰 모드, 2 - 검색 모드: , 3 - 프로그램 종료: 1
____________________________________________________
1번: 해당 동 전체 뷰 모드로 설정되었습니다.

신촌동의 음식점 분포입니다



모드를 골라주세요: 1 - 해당 동 전체 뷰 모드, 2 - 검색 모드: , 3 - 프로그램 종료: 3

이용해 주셔서 감사합니다
